# 6. Guess department city

In this kernel, I will try guessing the city of each department. Hmm!

In [1]:
%matplotlib inline

from cpe_help import Department
from cpe_help.tiger import get_tiger
from cpe_help.util.crs import equal_area_from_geodf

In [2]:
tiger = get_tiger()
dept = Department.sample()

In [3]:
uac = tiger.load_urban_areas()
police = dept.load_preprocessed_shapefile()
police = police.to_crs(uac.crs)

In [4]:
# check if name can be split
# Ok!

uac['NAME10'].apply(lambda x: x.count(',')).value_counts()

1    3601
Name: NAME10, dtype: int64

In [5]:
uac = uac[uac.intersects(police.unary_union)]
uac.head()

,UACE10,GEOID10,NAME10,NAMELSAD10,LSAD10,MTFCC10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
1593,09271,09271,"Boston, MA--NH--RI","Boston, MA--NH--RI Urbanized Area",75,G3500,U,S,4852285339,202265876,+42.3731315,-071.1407078,"(POLYGON ((-71.64691599999999 42.644126, -71.6..."


In [6]:
proj = equal_area_from_geodf(uac)
uac = uac.to_crs(proj)
police = police.to_crs(proj)

In [7]:
idx = uac.intersection(police.unary_union).area.idxmax()  # stil works with points (if only one city intersect)
idx

1593

In [8]:
city_row = uac.loc[idx]
city_row

UACE10                                                    09271
GEOID10                                                   09271
NAME10                                       Boston, MA--NH--RI
NAMELSAD10                    Boston, MA--NH--RI Urbanized Area
LSAD10                                                       75
MTFCC10                                                   G3500
UATYP10                                                       U
FUNCSTAT10                                                    S
ALAND10                                              4852285339
AWATER10                                              202265876
INTPTLAT10                                          +42.3731315
INTPTLON10                                         -071.1407078
geometry      (POLYGON ((-36146.84329126229 25088.2234472110...
Name: 1593, dtype: object

In [9]:
city_name = city_row['NAME10'].split(',')[0]
city_name

'Boston'

And that's the name we were looking for...